In [1]:
import numpy as np

# Choose the "true" parameters.
m_true = -0.9594
b_true = 4.294
f_true = 0.534

# Generate some synthetic data from the model.
N = 50
x = np.sort(10*np.random.rand(N))
yerr = 0.1+0.5*np.random.rand(N)
y = m_true*x+b_true
y += np.abs(f_true*y) * np.random.randn(N)
y += yerr * np.random.randn(N)


[ 0.20643092  0.25525907  0.51971523  0.52592229  0.6585319   0.87785337
  1.19101533  1.20221203  1.26137652  1.27781371  1.95443071  2.08198866
  2.48103474  3.03383728  3.13965601  3.18945833  3.42076972  3.47101302
  3.53289761  3.67969027  3.92634522  3.94541044  4.48982469  4.75606849
  4.96517244  5.01860504  5.43775539  5.53333662  5.60630792  5.76481914
  6.06855954  6.51760944  6.53932076  6.63606694  6.69319346  6.7184045
  6.74388935  6.88672963  7.45461008  7.51114795  7.54400382  8.28490314
  8.78127474  8.84816826  9.17084754  9.21988319  9.35013419  9.37619183
  9.65625193  9.84058573]


In [2]:
A = np.vstack((np.ones_like(x), x)).T
C = np.diag(yerr * yerr)
cov = np.linalg.inv(np.dot(A.T, np.linalg.solve(C, A)))
b_ls, m_ls = np.dot(cov, np.dot(A.T, np.linalg.solve(C, y)))

In [3]:
def lnlike(theta, x, y, yerr):
    m, b, lnf = theta
    model = m * x + b
    inv_sigma2 = 1.0/(yerr**2 + model**2*np.exp(2*lnf))
    return -0.5*(np.sum((y-model)**2*inv_sigma2 - np.log(inv_sigma2)))

In [17]:
import scipy.optimize as op
nll = lambda *args: -lnlike(*args)
result = op.minimize(nll, [m_true, b_true, np.log(f_true)], args=(x, y, yerr))
1, 2, 3 = result["x"]

SyntaxError: can't assign to literal (<ipython-input-17-0c0490b32f40>, line 4)

In [26]:
print(result["x"])
print(np.array([1, 2, 3]))
print(lnf)

[-0.90266123  4.28827301 -0.62260373]
[1 2 3]
-0.694821044283


In [5]:
def lnprior(theta):
    m, b, lnf = theta
    if -5.0 < m < 0.5 and 0.0 < b < 10.0 and -10.0 < lnf < 1.0:
        return 0.0
    return -np.inf

In [6]:
def lnprob(theta, x, y, yerr):
    lp = lnprior(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + lnlike(theta, x, y, yerr)

In [7]:
ndim, nwalkers = 3, 100
pos = [result["x"] + 1e-4*np.random.randn(ndim) for i in range(nwalkers)]

In [8]:
import emcee
sampler = emcee.EnsembleSampler(nwalkers, ndim, lnprob, args=(x, y, yerr))

In [9]:
sampler.run_mcmc(pos, 500)

(array([[-0.84903182,  3.92082022, -0.67223414],
        [-0.86805036,  4.06309003, -0.4873408 ],
        [-1.11467115,  5.67478457, -0.58693049],
        [-0.85933436,  4.14080993, -0.6559906 ],
        [-0.8961574 ,  3.95176513, -0.6511853 ],
        [-0.91714149,  4.4953471 , -0.61601508],
        [-0.81012245,  3.95734525, -0.49159559],
        [-0.86637735,  3.89363353, -0.59417038],
        [-0.99055445,  4.70247172, -0.62391705],
        [-0.97876873,  4.72427874, -0.61678705],
        [-0.90881602,  4.35083955, -0.76657768],
        [-0.93752983,  4.51366055, -0.65683811],
        [-0.8361755 ,  3.82680604, -0.62298296],
        [-0.84155059,  3.95490565, -0.66222079],
        [-0.79700675,  3.84324394, -0.46154864],
        [-0.85171697,  3.91551041, -0.44001079],
        [-0.84029416,  3.92294291, -0.5282237 ],
        [-0.92230701,  4.25252756, -0.76252721],
        [-0.83768711,  3.95735359, -0.46385095],
        [-0.89464771,  4.11920879, -0.75920525],
        [-0.97572851

In [21]:
samples = sampler.chain[:, 50:, :].reshape((-1, ndim))